In [1]:
from datetime import datetime, date

import pandas as pd

from log_info import update_log_df


pd.set_option('display.max_colwidth', 200)

In [2]:
log_df: pd.DataFrame = update_log_df()

**Enter report dates**

**_______**

In [7]:
begin_date = date(year=2018, month=12, day=10)
end_date = date(year=2018, month=12, day=18)

**______**

In [6]:
log_df_filtered = log_df[(log_df['date'] >= begin_date) & (log_df['date'] <= end_date)]
report_columns = ['outer_request', 'resource_group', 'resource']
groupby_columns = ['outer_request', 'resource_group']
log_df_groped = log_df_filtered[report_columns].groupby(groupby_columns)
print(f'Download statistics from {str(begin_date)} to {str(end_date)}')
log_df_groped.count()

Download statistics from 2018-12-12 to 2018-12-18


resource
outer_request resource_group           
False         datasets              824
              deeppavlov_data      1825
              ecommerce_skill       424
              embeddings            428
              faq                   546
              lang_models            99
              vectorizer             53
True          datasets              211
              deeppavlov_data       541
              ecommerce_skill         4
              embeddings            219
              faq                    40
              lang_models            11
              ner                     2
              vectorizer              7
              yad                    12